In [ ]:
from HelmholtzSolver2D import *
from ExampleBoundaries import square

# Test Problem 1
# Dirichlet boundary condition with phi chose to be the same
# as a point-source at the center of the square.
#
solver = ExteriorHelmholtzSolver2D(square())

frequency = 400.0 # frequency [Hz]
k = frequency_to_wavenumber(frequency)

center_square = np.array([0.05, 0.05], dtype=np.float32)
boundary_condition = solver.dirichlet_boundary_condition()
centers = solver.geometry.centers()
for i in range(centers.shape[0]):
    r = norm(centers[i, :] - center_square)
    boundary_condition.f[i] = 1j * hankel1(0, k*r) / 4.0

boundary_incidence = BoundaryIncidence(solver.len())
boundary_incidence.phi.fill(0.0)
boundary_incidence.v.fill(0.0)

exterior_points = np.array([[0.0000,  0.1500],
                           [0.0500,  0.1500],
                           [0.1000,  0.1500],
                           [0.0500, -0.1000]], dtype=np.float32)

exterior_incident_phi = np.zeros(exterior_points.shape[0], dtype=np.complex64)

boundary_solution = solver.solve_boundary(k, boundary_condition, boundary_incidence)
sample_solution = boundary_solution.solve_samples(exterior_incident_phi, exterior_points)
print("\n\nTest Problem 1")
print("==============\n")
print(boundary_solution)
print(sample_solution)

# Test Problem 2
# Neumann boundary condition with v chosen to be the same
# as a point-source at the center of the square.
#
boundary_condition = solver.neumann_boundary_condition()

center_square = np.array([0.05, 0.05], dtype=np.float32)
for i in range(centers.shape[0]):
    x = centers[i, :]
    r = x - center_square
    R = norm(r)
    if x[0] <= 1e-7:
        drbdn = -r[0]/R
    elif x[0] >= 0.1 - 1e-7:
        drbdn = r[0]/R
    elif x[1] <= 1e-7:
        drbdn = -r[1]/R
    elif x[1] >= 0.1 - 1e-7:
        drbdn = r[1]/R
    else:
        assert False, "r = ({}, {}) not properly handled.".format(r[0], r[1])
    boundary_condition.f[i] = -1j * k * drbdn * hankel1(1, k*R) / 4.0

boundary_solution = solver.solve_boundary(k, boundary_condition, boundary_incidence)
sample_solution = boundary_solution.solve_samples(exterior_incident_phi, exterior_points)
print("\n\nTest Problem 2")
print("==============\n")
print(boundary_solution)
print(sample_solution)

# Test Problem 3
# Neumann boundary condition with v=0 on the full boundary.
# The 400Hz field in this case is generated by a point source
# at (0.5, 0.25)
boundary_condition = solver.neumann_boundary_condition()

source_point = np.array([0.5, 0.25], dtype=np.float32)
print("source_point = ", source_point)
for i in range(centers.shape[0]):
    x = centers[i, :]
    r = x - source_point
    R = norm(r)
    boundary_incidence.phi[i] = 1j * hankel1(0, k*R) / 4.0
    if x[0] <= 1e-7:
        drbdn = -r[0]/R
    elif x[0] >= 0.1 - 1e-7:
        drbdn = r[0]/R
    elif x[1] <= 1e-7:
        drbdn = -r[1]/R
    elif x[1] >= 0.1 - 1e-7:
        drbdn = r[1]/R
    else:
        assert False, "r = ({}, {}) not properly handled.".format(r[0], r[1])
    boundary_incidence.v[i] = -1j * k * drbdn * hankel1(1, k*R) / 4.0

boundary_solution = solver.solve_boundary(k, boundary_condition, boundary_incidence)

for i in range(exterior_points.shape[0]):
    R = norm(exterior_points[i, :] - source_point)
    exterior_incident_phi[i] = 1j * hankel1(0, k*R) / 4.0
    
sample_solution = boundary_solution.solve_samples(exterior_incident_phi, exterior_points)
print("\n\nTest Problem 3")
print("==============\n")
print(boundary_solution)
print(sample_solution)

Copyright (C) 2017&ndash;2019 Frank Jargstorff

This file is part of the AcousticBEM library.

AcousticBEM is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

AcousticBEM is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with AcousticBEM. If not, see http://www.gnu.org/licenses/.